In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:

import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from PIL import Image


In [26]:

# Setting up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Parameters
batch_size = 64
num_workers = 8
pin_memory = True
training_epochs = 10
learning_rate = 0.001

# Data loading and transformation setup
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset and split into training and testing
def load_split_data(root_dir, valid_size=0.2):
    dataset = ImageFolder(root=root_dir, transform=data_transforms)
    classes = dataset.classes
    class_to_idx = dataset.class_to_idx
    train_idx, test_idx = train_test_split(list(range(len(dataset))), test_size=valid_size, stratify=dataset.targets)

    # Calculate class weights and prepare the sampler
    class_counts = torch.bincount(torch.tensor(dataset.targets))
    class_weights = 1. / (class_counts.float() + 1e-6)
    sample_weights = class_weights[dataset.targets]
    sampler = WeightedRandomSampler(sample_weights[train_idx], num_samples=len(train_idx), replacement=True)

    train_subset = torch.utils.data.Subset(dataset, train_idx)
    test_subset = torch.utils.data.Subset(dataset, test_idx)
    train_loader = DataLoader(train_subset, batch_size=batch_size, sampler=sampler, num_workers=num_workers, pin_memory=pin_memory)
    test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)
    return train_loader, test_loader, classes

train_loader, test_loader, classes = load_split_data('/content/drive/MyDrive/JPEGImages')


In [27]:
# Model definition
def create_model(num_classes):
    model = models.resnet50(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, num_classes),
        nn.LogSoftmax(dim=1)
    )
    return model.to(device)

model = create_model(len(classes))

# Loss and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)

# Enable mixed precision training
scaler = torch.cuda.amp.GradScaler()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [28]:
# Training the model
def train_model():
    model.train()
    for epoch in range(training_epochs):
        start_time = time.time()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            if i % 20 == 19:
                print(f'Epoch {epoch + 1}, Step {i + 1}, Loss: {running_loss / 20}')
                running_loss = 0.0
        end_time = time.time()
        print(f'Epoch {epoch + 1} completed in {end_time - start_time:.2f} seconds')

In [29]:

# Model evaluation
def evaluate_model():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the model on the test images: {100 * correct / total}%')



In [30]:
train_model()
evaluate_model()


Epoch 1, Step 20, Loss: 3.30799024105072
Epoch 1, Step 40, Loss: 1.9609145820140839
Epoch 1, Step 60, Loss: 1.2360934972763062
Epoch 1, Step 80, Loss: 0.9123417884111404
Epoch 1, Step 100, Loss: 0.783375883102417
Epoch 1, Step 120, Loss: 0.7305864036083222
Epoch 1 completed in 27.54 seconds
Epoch 2, Step 20, Loss: 0.6360910266637803
Epoch 2, Step 40, Loss: 0.6424988746643067
Epoch 2, Step 60, Loss: 0.5469781264662743
Epoch 2, Step 80, Loss: 0.4926511198282242
Epoch 2, Step 100, Loss: 0.5562603548169136
Epoch 2, Step 120, Loss: 0.4854041576385498
Epoch 2 completed in 27.37 seconds
Epoch 3, Step 20, Loss: 0.4865072265267372
Epoch 3, Step 40, Loss: 0.43984404355287554
Epoch 3, Step 60, Loss: 0.41095187067985534
Epoch 3, Step 80, Loss: 0.4568556137382984
Epoch 3, Step 100, Loss: 0.4341370686888695
Epoch 3, Step 120, Loss: 0.3708513617515564
Epoch 3 completed in 27.19 seconds
Epoch 4, Step 20, Loss: 0.38141823187470436
Epoch 4, Step 40, Loss: 0.39135692194104194
Epoch 4, Step 60, Loss: 0.37